In [1]:
from feature_store import FeatureStoreClient
import polars as pl
import numpy as np

In [2]:
client = FeatureStoreClient(host="localhost", port=6379)

# Create some tables and insert data. 

## supported data types: i8, i16, i32, i64, f32, f64, utf-8

In [3]:
n_items = 1000
data = pl.DataFrame({
    "s1": np.arange(n_items, dtype=np.int32),
    "s2": ["a"] * n_items, 
    "s3": np.random.normal(0, 10, n_items),
    "s4": np.random.normal(0, 10, n_items),
    "s5": np.random.normal(0, 10, n_items),
    "s6": np.random.normal(0, 10, n_items)
})

client.create_table(
    table_name="features1",
    columns=["s1", "s2", "s3", "s4", "s5", "s6"], 
    primary_key=["s1", "s2"], 
    data_types=["i32","utf8", "f32", "f32", "f32", "f32"]
)
client.create_table(
    table_name="features2",
    columns=["s1", "s2", "s3", "s4", "s5", "s6"], 
    primary_key=["s1", "s2"], 
    data_types=["i32","utf8", "f32", "f32", "f32", "f32"]
)
client.create_table(
    table_name="features3",
    columns=["s1", "s2", "s3", "s4", "s5", "s6"], 
    primary_key=["s1", "s2"], 
    data_types=["i32","utf8", "f32", "f32", "f32", "f32"]
)

client.insert(table_name="features1", data=data)
client.insert(table_name="features2", data=data)
client.insert(table_name="features3", data=data)

Inserting s3 feature
Inserting s4 feature
Inserting s5 feature
Inserting s6 feature
Inserting s3 feature
Inserting s4 feature
Inserting s5 feature
Inserting s6 feature
Inserting s3 feature
Inserting s4 feature
Inserting s5 feature
Inserting s6 feature


# Query some records for an imaginary ml model

In [4]:
n_items = 200
keys = pl.DataFrame({"s1": np.arange(n_items), "s2": ["a"] * n_items})

In [5]:
keys

s1,s2
i64,str
0,"""a"""
1,"""a"""
2,"""a"""
3,"""a"""
4,"""a"""
5,"""a"""
6,"""a"""
7,"""a"""
8,"""a"""


In [7]:
client.get_tables()

['features3', 'features1', 'features2']

In [6]:
client.describe_table(table_name="features1")

{'s1': 'i32',
 's2': 'utf8',
 's3': 'f32',
 's4': 'f32',
 's5': 'f32',
 's6': 'f32',
 'primary_key': ['s1', 's2']}

## client knows nothing about your tables, so it is neccessary to load tables' metadata to handle selection properly (it saves some time because we do not need to additionally query redis / keydb for metadata during the 'select' requests). If we create tables client will automatically save metadata in the cache, so there is no need loading it.

In [8]:
# client.load_table_meta(table_name="features1")
# client.load_table_meta(table_name="features3")
# client.load_table_meta(table_name="features2")

In [12]:
data = client.select(
    table_keys={"features1": keys, "features2": keys, "features3": keys}, 
    feature_columns={"features1": ["s3", "s4", "s5", "s6"], "features2": ["s3", "s4", "s5"], "features3": ["s3", "s4", "s5", "s6"]}
)
data

{'features1': shape: (200, 4)
 ┌────────────┬────────────┬────────────┬────────────┐
 │ s3         ┆ s4         ┆ s5         ┆ s6         │
 │ ---        ┆ ---        ┆ ---        ┆ ---        │
 │ f32        ┆ f32        ┆ f32        ┆ f32        │
 ╞════════════╪════════════╪════════════╪════════════╡
 │ -9.392688  ┆ 7.926713   ┆ -22.057686 ┆ -14.6161   │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -15.488579 ┆ 1.7225     ┆ -0.089746  ┆ -8.561509  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -1.953087  ┆ -13.176362 ┆ -18.394203 ┆ 17.872992  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -2.784775  ┆ -4.286248  ┆ -1.530366  ┆ -7.321002  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ ...        ┆ ...        ┆ ...        ┆ ...        │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ 4.476875   ┆ -5.416066  ┆ -2.856553  ┆ 0.258304   │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ 2.005133   ┆ -0.1735    ┆ 19.711

In [10]:
%time client.select(table_keys={"features1": keys, "features2": keys, "features3": keys}, feature_columns={"features1": ["s3", "s4", "s5", "s6"], "features2": ["s3", "s4", "s5"], "features3": ["s3", "s4", "s5", "s6"]})

CPU times: user 6.27 ms, sys: 4.62 ms, total: 10.9 ms
Wall time: 20.5 ms


{'features1': shape: (200, 4)
 ┌────────────┬────────────┬────────────┬────────────┐
 │ s3         ┆ s4         ┆ s5         ┆ s6         │
 │ ---        ┆ ---        ┆ ---        ┆ ---        │
 │ f32        ┆ f32        ┆ f32        ┆ f32        │
 ╞════════════╪════════════╪════════════╪════════════╡
 │ -9.392688  ┆ 7.926713   ┆ -22.057686 ┆ -14.6161   │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -15.488579 ┆ 1.7225     ┆ -0.089746  ┆ -8.561509  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -1.953087  ┆ -13.176362 ┆ -18.394203 ┆ 17.872992  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ -2.784775  ┆ -4.286248  ┆ -1.530366  ┆ -7.321002  │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ ...        ┆ ...        ┆ ...        ┆ ...        │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ 4.476875   ┆ -5.416066  ┆ -2.856553  ┆ 0.258304   │
 ├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
 │ 2.005133   ┆ -0.1735    ┆ 19.711